# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [14]:
%pip install numpy pandas matplotlib scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [15]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

# Install missing packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


In [16]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [ ]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        X_train_pca = X_train.copy()
        X_val_pca = X_val.copy()

        if hyperparams['pca_standardize']:
            scaler_pca = StandardScaler()
            X_train_pca = scaler_pca.fit_transform(X_train_pca)
            X_val_pca = scaler_pca.transform(X_val_pca)

        pca = PCA(n_components=hyperparams['pca_components'])
        X_train_pca = pca.fit_transform(X_train_pca)
        X_val_pca = pca.transform(X_val_pca)
    else:
        X_train_pca = X_train.copy()
        X_val_pca = X_val.copy()


    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train_pca = scaler.fit_transform(X_train_pca)
        X_val_pca = scaler.transform(X_val_pca)
    
    # Aggiunge il termine costante ai dati
    X_train_ = np.c_[X_train_pca, np.ones(X_train_pca.shape[0])]
    X_val_ = np.c_[X_val_pca, np.ones(X_val_pca.shape[0])]



    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        lmbd = hyperparams['reg_lambda']
        I = np.eye(X_train_.shape[1])
        theta = np.linalg.inv(X_train_.T @ X_train_ + lmbd * I) @ X_train_.T @ y_train
        y_train_pred = X_train_ @ theta
        y_val_pred = X_val_ @ theta
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_val = mean_absolute_error(y_val, y_val_pred)
    else:
        model = LinearRegression()
        model.fit(X_train_, y_train)
        y_train_pred = model.predict(X_train_)
        y_val_pred = model.predict(X_val_)
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_val = mean_absolute_error(y_val, y_val_pred)
        theta = None  

    return mae_train, mae_val, theta

# Dividi il dataset in training e test set
train_fraction = 0.6
validation_fraction = 0.2
test_fraction = 0.2

num_train = int(train_fraction * X.shape[0])
num_validation = int(validation_fraction * X.shape[0])
X_train = X[:num_train]
y_train = y[:num_train]


X_validation = X[num_train:num_train + num_validation]
y_validation = y[num_train:num_train + num_validation]

X_test = X[num_train + num_validation:]
y_test = y[num_train + num_validation:]


# Trova la configurazione di iperparametri migliore
def trova_migliore_configurazione(X_train, y_train, X_val, y_val, configs):
    best_config = None
    best_mae_val = float('inf')
    best_mae_train = None
    best_theta = None
    tutte_le_configurazioni = []

    for config in configs:
        mae_train, mae_val, theta = pipeline(X_train, y_train, X_val, y_val, config)
        tutte_le_configurazioni.append({
            "config": config,
            "mae_train": mae_train,
            "mae_val": mae_val
        })
        if mae_val < best_mae_val:
            best_mae_val = mae_val
            best_config = config
            best_mae_train = mae_train
            best_theta = theta

    return best_config, best_mae_train, best_mae_val, best_theta, tutte_le_configurazioni


best_config, best_mae_train, best_mae_val, best_theta, tutte_le_configurazioni = trova_migliore_configurazione(
    X_train, y_train, X_validation, y_validation, configs
)
# Riallena il modello sul training set completo
X_trainval = np.concatenate([X_train, X_validation])
y_trainval = np.concatenate([y_train, y_validation])
mae_train_final, _, theta_final = pipeline(X_trainval, y_trainval, X_test, y_test, best_config)


# Calcola il MAE sul test set
if theta_final is not None:
    X_test_proc = X_test.copy()
    if best_config['use_pca'] and best_config['pca_standardize']:
        X_test_proc = StandardScaler().fit_transform(X_test_proc)
    if best_config['use_pca']:
        X_test_proc = PCA(n_components=best_config['pca_components']).fit_transform(X_test_proc)
    if best_config['data_standardize']:
        X_test_proc = StandardScaler().fit_transform(X_test_proc)
    X_test_proc = np.c_[X_test_proc, np.ones(X_test_proc.shape[0])]
    y_test_pred = X_test_proc @ theta_final
    mae_test = mean_absolute_error(y_test, y_test_pred)


# Stampa  risultati
print("Migliore configurazione:", best_config)
print("MAE di validation:", best_mae_val)
print("MAE di train:", mae_train_final)
print("MAE di test:", mae_test)



altre = [conf for conf in tutte_le_configurazioni if conf["config"] != best_config][:5]
for i, conf in enumerate(altre, 1):
    print(f"\n#{i}")
    print("Configurazione:", conf["config"])
    print("MAE train:", conf["mae_train"])
    print("MAE val:", conf["mae_val"])


Migliore configurazione: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE di validation: 0.5872605946947294
MAE di train: 0.5942737135129478
MAE di test: 0.5222475306617327

#1
Configurazione: {'use_pca': True, 'pca_standardize': True, 'pca_components': 5, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 10}
MAE train: 0.6405745178522934
MAE val: 0.632868570297251

#2
Configurazione: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 1}
MAE train: 0.6034411661268437
MAE val: 0.5901691196351071

#3
Configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE train: 0.6031219835420769
MAE val: 0.5874348774687382

#4
Configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 3,

c:\Users\damia\OneDrive\Desktop\HomeWork1ML\.conda\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
